Exp 1: baseline 'fat' model with 2 heads (via group conv), l1 loss for rotation and translation, 10 epochs

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
import torch
from torch.utils.data import DataLoader

from flow_regressors.flow_dataset import FlowDataset
from flow_regressors.regressors import DensePoseRegressorV1

from training.train_flow import train
from training.loss_pose import LossPose

from tqdm.auto import tqdm

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Data

In [4]:
train_data = FlowDataset(
    root_path='/home/project/data_sample/flow/train',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz'
    )

train_loader = DataLoader(train_data, batch_size=128, shuffle=True, drop_last=True, pin_memory=True, num_workers=0)

val_data = FlowDataset(
    root_path='/home/project/data_sample/flow/val',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz'
    )

val_loader = DataLoader(val_data, batch_size=128, shuffle=False, drop_last=False, pin_memory=True, num_workers=0)

### 2. Configuration

In [5]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

config = dict(
    general = dict(
        experiment_name='FLOW_TO_POSE_EXP_1',
        device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
        
        n_epochs=10,
        n_steps_per_epoch=len(train_loader.dataset) // train_loader.batch_size,
        n_accum_steps=1,
        batch_size=train_loader.batch_size,
        
        swa=False,
        n_epochs_swa=None,
        n_steps_between_swa_updates=None,
          
        repeat_val_epoch=1,
        repeat_save_epoch=5,
          
        model_save_path='../src/weights/FLOW_TO_POSE_EXP_1'
    )
)

### 3. Model

In [6]:
# Inference on (120, 160) input: 1.27 ms ± 2.88 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

model = DensePoseRegressorV1()
model.to(config['general']['device']);

In [18]:
# data = train_data[0]
# data['flow'] = torch.from_numpy(data['flow']).cuda().unsqueeze(0) 
# data['K_q'] = torch.from_numpy(data['K_q']).cuda().unsqueeze(0) 
# data['K_s'] = torch.from_numpy(data['K_s']).cuda().unsqueeze(0) 
# model.eval();

In [19]:
# %%timeit

# with torch.no_grad():
#     q, t = model(
#         data['flow'], data['K_q'], data['K_s'], (0.25, 0.25), (0.25, 0.25)
#        )

1.27 ms ± 2.88 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### 4. Loss, optimizer, scheduler

In [7]:
train_loss = LossPose(agg_type='mean')
val_loss = LossPose()

In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-6)
config['optimizer'] = optimizer.__dict__['defaults']

In [9]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                   step_size=2,
                   gamma=0.8)

### 6. Experiment

In [10]:
train(model, optimizer, scheduler, train_loss, val_loss, train_loader, val_loader, config, **config['general'])

wandb: Currently logged in as: kovanic. Use `wandb login --relogin` to force relogin


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 333/333 [04:46<00:00,  1.16it/s]


epoch 0: train loss=-5.862944286570147, val loss(q)=0.21294089932219712, val loss(t)=1.064414878185326


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 333/333 [04:46<00:00,  1.16it/s]


epoch 1: train loss=-9.105586811986646, val loss(q)=0.17211473200545077, val loss(t)=0.9427362136708082


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 333/333 [04:45<00:00,  1.17it/s]


epoch 8: train loss=-12.15370073967777, val loss(q)=0.10019569401211381, val loss(t)=0.7637798010005584


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 333/333 [04:45<00:00,  1.17it/s]

epoch 9: train loss=-12.362911907160871, val loss(q)=0.09789351973549362, val loss(t)=0.7504636159786227


Train batch loss,█▅▅▄▄▄▄▄▃▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁
Train loss epoch,█▅▄▃▃▂▂▁▁▁
Val loss epoch(q),█▆▃▂▂▂▂▁▁▁
Val loss epoch(t),█▅▆▅▃▂▂▁▁▁
Train batch loss,-12.107
Train loss epoch,-12.36291
Val loss epoch(q),0.09789
Val loss epoch(t),0.75046
